# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt','wordnet','stopwords'])
import re #first remove punctuation with a regular expression
from nltk.tokenize import word_tokenize #second tokenize words word
from nltk.corpus import stopwords #third remove stop words
from nltk.stem import WordNetLemmatizer #fourth lemmatization to reduce words to its normalized form

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/DuongNguyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/DuongNguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/DuongNguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Categorized', con=engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Define feature and target variables X and Y
X = df.message.values
Y = df.drop(['id','message','original', 'genre'], axis=1).values

In [5]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [6]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    #normalizae case and remove punctuation using regular expression
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #remove stop words using list comprehension
    stop_words = stopwords.words('english')
    tokens = [tok for tok in tokens if tok not in stop_words]
    
    #initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #iterate through each token and lemmatize, remove leading/trailing white spaces
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [8]:
#test out function
for message in X[:5]:
    tokens = tokenize(message)
    print(message, '\n')
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti 

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over 

['hurricane'] 

Looking for someone but no name 

['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately. 

['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight 

['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
     ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [11]:
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# Print out the column names in the classification report
col_names = df.drop(['id','message','original', 'genre'], axis=1).columns
for i, col in enumerate(col_names):
    print("Classification report for", col)
    print(classification_report(Y_test[:, i], Y_pred[:, i]))

Classification report for related
              precision    recall  f1-score   support

           0       0.71      0.43      0.53      1514
           1       0.85      0.95      0.89      5040

    accuracy                           0.83      6554
   macro avg       0.78      0.69      0.71      6554
weighted avg       0.82      0.83      0.81      6554

Classification report for request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5409
           1       0.85      0.51      0.63      1145

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.79      6554
weighted avg       0.89      0.90      0.89      6554

Classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6533
           1       0.00      0.00      0.00        21

    accuracy                           1.00      6554
   macro avg       0.50     

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6210
           1       0.53      0.03      0.05       344

    accuracy                           0.95      6554
   macro avg       0.74      0.51      0.51      6554
weighted avg       0.93      0.95      0.92      6554

Classification report for direct_report
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      5249
           1       0.82      0.36      0.50      1305

    accuracy                           0.86      6554
   macro avg       0.84      0.67      0.71      6554
weighted avg       0.85      0.86      0.83      6554



/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

In [24]:
print(classification_report(Y_test, Y_pred, zero_division = 0, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.89      5040
               request       0.85      0.51      0.63      1145
                 offer       0.00      0.00      0.00        21
           aid_related       0.76      0.69      0.72      2741
          medical_help       0.57      0.10      0.16       482
      medical_products       0.79      0.09      0.16       327
     search_and_rescue       0.75      0.03      0.06       185
              security       0.50      0.01      0.02       123
              military       0.46      0.05      0.10       201
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.45      0.59       428
                  food       0.87      0.62      0.72       737
               shelter       0.85      0.40      0.54       601
              clothing       1.00      0.08      0.15       109
                 money       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fa33456c0d0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fa33456c0d0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [14]:
def build_model():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
    parameters = {
    #'vect__ngram_range':((1,1),(1,2),(2,2)),
    #'vect__max_df':(0.5, 0.75, 1.0),
    #'vect__max_features':(None, 5000, 10000),
    'tfidf__use_idf':(True, False),
    'clf__estimator__n_estimators':(10, 20, 30)
    #'clf__estimator__min_samples_split':(2, 3, 4)
}
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [15]:
model = build_model()
model.fit(X_train, Y_train)
Y_pred_model = model.predict(X_test)

In [16]:
def display_results(cv, Y_test, Y_pred_model):
    print("best parameters:", cv.best_params_)

In [17]:
display_results(model, Y_test, Y_pred_model)

best parameters: {'clf__estimator__n_estimators': 30, 'tfidf__use_idf': True}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
# Print out classification report for tuned model
#col_names = df.drop(['id','message','original', 'genre'], axis=1).columns
for i, col in enumerate(col_names):
    print("Classification report for tuned model,", col)
    print(classification_report(Y_test[:, i], Y_pred_model[:, i]))

Classification report for tuned model, related
              precision    recall  f1-score   support

           0       0.69      0.43      0.53      1514
           1       0.85      0.94      0.89      5040

    accuracy                           0.82      6554
   macro avg       0.77      0.69      0.71      6554
weighted avg       0.81      0.82      0.81      6554

Classification report for tuned model, request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5409
           1       0.83      0.46      0.60      1145

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554

Classification report for tuned model, offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6533
           1       0.00      0.00      0.00        21

    accuracy                           1.00

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6439
           1       0.89      0.07      0.13       115

    accuracy                           0.98      6554
   macro avg       0.94      0.53      0.56      6554
weighted avg       0.98      0.98      0.98      6554

Classification report for tuned model, other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6210
           1       0.57      0.03      0.07       344

    accuracy                           0.95      6554
   macro avg       0.76      0.52      0.52      6554
weighted avg       0.93      0.95      0.93      6554

Classification report for tuned model, direct_report
              precision    recall  f1-score   support

           0       0.86      0.98      0.91      5249
           1       0.79      0.35      0.49      1305

    accuracy                           0.85      6554
   macro avg       0.8

/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

In [27]:
print(classification_report(Y_test, Y_pred_model, zero_division = 0, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      5040
               request       0.83      0.46      0.60      1145
                 offer       0.00      0.00      0.00        21
           aid_related       0.75      0.66      0.70      2741
          medical_help       0.54      0.09      0.15       482
      medical_products       0.72      0.08      0.14       327
     search_and_rescue       0.89      0.04      0.08       185
              security       0.40      0.02      0.03       123
              military       0.57      0.06      0.12       201
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.40      0.54       428
                  food       0.84      0.54      0.66       737
               shelter       0.87      0.34      0.49       601
              clothing       1.00      0.06      0.10       109
                 money       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# Try using KNeighborsClassifier model
from sklearn.neighbors import KNeighborsClassifier
KNpipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('kneigh', MultiOutputClassifier(KNeighborsClassifier()))
])
KNpipeline.fit(X_train, Y_train)
Y_pred_KN = KNpipeline.predict(X_test)

In [20]:
#print out classification report
#col_names = df.drop(['id','message','original', 'genre'], axis=1).columns
for i, col in enumerate(col_names):
    print("Classification report for KNeigh model,", col)
    print(classification_report(Y_test[:, i], Y_pred_KN[:, i]))

Classification report for KNeigh model, related
              precision    recall  f1-score   support

           0       0.71      0.13      0.22      1514
           1       0.79      0.98      0.88      5040

    accuracy                           0.79      6554
   macro avg       0.75      0.56      0.55      6554
weighted avg       0.77      0.79      0.72      6554

Classification report for KNeigh model, request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      5409
           1       0.83      0.09      0.16      1145

    accuracy                           0.84      6554
   macro avg       0.83      0.54      0.53      6554
weighted avg       0.84      0.84      0.78      6554

Classification report for KNeigh model, offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6533
           1       0.00      0.00      0.00        21

    accuracy                           1

              precision    recall  f1-score   support

           0       0.81      1.00      0.89      5249
           1       0.81      0.06      0.11      1305

    accuracy                           0.81      6554
   macro avg       0.81      0.53      0.50      6554
weighted avg       0.81      0.81      0.74      6554



/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/DuongNguyen/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

In [28]:
print(classification_report(Y_test, Y_pred_KN, zero_division = 0, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.88      5040
               request       0.83      0.09      0.16      1145
                 offer       0.00      0.00      0.00        21
           aid_related       0.79      0.04      0.07      2741
          medical_help       0.00      0.00      0.00       482
      medical_products       0.50      0.00      0.01       327
     search_and_rescue       0.00      0.00      0.00       185
              security       0.00      0.00      0.00       123
              military       0.00      0.00      0.00       201
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.05      0.09       428
                  food       0.81      0.07      0.12       737
               shelter       0.81      0.04      0.08       601
              clothing       1.00      0.04      0.07       109
                 money       0.67      

### 9. Export your model as a pickle file

In [35]:
import pickle
with open('train_classifier.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.